In [ ]:
# TODO HACK deleteme
import sys
import os
PROJECT_DIR = "/home/usr1/jboyle/neuro/sana-fe"
sys.path.insert(0, os.path.join(PROJECT_DIR))

In [ ]:
#%pip install --extra-index-url https://test.pypi.org/simple/ sanafe==0.0.3
%pip install pyyaml
!wget -nc https://raw.githubusercontent.com/SLAM-Lab/SANA-FE/cpp/arch/loihi.yaml
!wget -nc https://raw.githubusercontent.com/SLAM-Lab/SANA-FE/cpp/tutorial/dvs_challenge.npz
import sanafe
import yaml

In [ ]:
arch = sanafe.load_arch("loihi.yaml")
network = sanafe.Network()

# Load the convolutional kernel weights, thresholds and input biases from file.
#  If using the Docker container, this file is included in the image.
#  Otherwise, this file is also hosted on Google Drive and can be downloaded
#  prior to running this script
import numpy as np
try:
    snn = np.load("dvs_challenge.npz")
except FileNotFoundError as exc:
    print(exc)
    print("""
To run this challenge, you need to download the network kernel weights: dvs_challenge.npz, to the tutorial directory.
These weights are hosted online on a shared Google Drive. To download the file with a in Linux run the command:

wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1WkbJZFasTe-v8vTYXrUaz_1e-p_xHMEj' -O tutorial/dvs_challenge.npz

Or go directly to the drive at: https://drive.google.com/drive/folders/1GzjXAFouakm3b6GcFIHsw67H8t6l3BtY?usp=drive_link
          """)
    exit()

# Convert the DVS gesture categorization model to SANA-FE's SNN format
thresholds = snn["thresholds"]
layer0 = network.create_neuron_group("in", 1024, {"threshold": thresholds[0]})
layer1 = network.create_neuron_group("conv1", 3600, {"threshold": thresholds[1]})
layer2 = network.create_neuron_group("conv2", 5408, {"threshold": thresholds[2]})
layer3 = network.create_neuron_group("conv3", 7744, {"threshold": thresholds[3]})
layer4 = network.create_neuron_group("conv4", 891, {"threshold": thresholds[4]})
layer5 = network.create_neuron_group("dense1", 11, {"threshold": thresholds[5]})

# Now define connectivity
print(snn["conv1"].shape)
print(snn["conv1"].flatten())
# TODO: is the shape matching up with what I expect - should be channels last?
# I liked the old interface better, so maybe I should support another level
#  which basically allows us to create layers i.e. creates the neurons and connections
#  in one go. Can have a deep learning python file
layer0.connect_neurons_conv2d(layer1, {"w": snn["conv1"].flatten()}, 32, 32, 1, 3, 3, kernel_count=16, stride_width=2, stride_height=2)
layer1.connect_neurons_conv2d(layer2, {"w": snn["conv2"].flatten()}, 15, 15, 16, 3, 3, kernel_count=32)
layer2.connect_neurons_conv2d(layer3, {"w": snn["conv3"].flatten()}, 13, 13, 32, 3, 3, kernel_count=64)
layer3.connect_neurons_conv2d(layer4, {"w": snn["conv4"].flatten()}, 11, 11, 64, 3, 3, kernel_count=11)
layer4.connect_neurons_dense(layer5, {"w": snn["dense1"].flatten()})

biases = snn["inputs"]
# Finally set up the inputs
for n, b in zip(layer0.neurons, biases):
    n.configure(model_parameters={"bias": b})


In [ ]:
# Map the SNN to Loihi cores. Each layer is represented by its own neuron group
### CHANGE THESE LINES TO TEST DIFFERENT MAPPINGS ###
#sim.map_neuron_group_to_cores(layer0, arch, 1)
#sim.map_neuron_group_to_cores(layer1, arch, 4)
#sim.map_neuron_group_to_cores(layer2, arch, 16)
#sim.map_neuron_group_to_cores(layer3, arch, 16)
#sim.map_neuron_group_to_cores(layer4, arch, 4)
#sim.map_neuron_group_to_cores(layer5, arch, 1)
### END OF LINES TO CHANGE ###

# TODO: get something like the above code again that's simpler
for n in layer0.neurons:
    n.map_to_core(arch.tiles[0].cores[0])

for i, n in enumerate(layer1.neurons): # across 4 cores
    core = i % 4
    tile = core // 4
    core_offset = core % 4
    n.map_to_core(arch.tiles[2 + tile].cores[core_offset])

for i, n in enumerate(layer2.neurons): # across 16 cores
    total_neurons = len(layer2.neurons)
    core = i % 16
    tile = core // 4
    core_offset = core % 4
    n.map_to_core(arch.tiles[4 + tile].cores[core_offset])

for i, n in enumerate(layer3.neurons): # across 16 cores
    core = i % 16
    tile = core // 4
    core_offset = core % 4
    n.map_to_core(arch.tiles[8 + tile].cores[core_offset])

for i, n in enumerate(layer4.neurons): # Across 4 cores
    core = i % 4
    tile = core // 4
    core_offset = core % 4
    n.map_to_core(arch.tiles[12 + tile].cores[core_offset])

for n in layer5.neurons:
    n.map_to_core(arch.tiles[16].cores[0])

In [ ]:
chip = sanafe.SpikingChip(arch)
chip.load(network)
# Run the network you just generated on Loihi
# Comment out this line if you want to stop the simulations running
results = chip.sim(10)
expected_firing_neurons = 367770
print(results)
if results["neurons_fired"] != expected_firing_neurons:
    print(f"Error: The total number of neurons spiking was "
          f"{results['neurons_fired']}, "
          f"should be {expected_firing_neurons}")
    print("Somehow you may have changed the functional behavior of the SNN")
    raise RuntimeError

energy_delay_product = results["energy"] * results["sim_time"]
print(f"Energy-Delay product: {energy_delay_product}")